In [347]:
from google.cloud.bigquery.client import Client
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import datetime

In [10]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='../bqcreds.json'
bq_client = Client()

In [19]:
sql = """
Select 
    * 
FROM 
    stock_data.stock_stats 
ORDER BY
    date desc
"""
data = bq_client.query(sql)
data

In [21]:
data = data.to_dataframe()

In [22]:
data

,date,symbol,sharesShort,bookValue,trailingEps,forwardEps,pegRatio,previousClose,open,currentPrice,...,bid,ask,bidSize,askSize,strongBuy,buy,hold,sell,strongSell,recommendationMean
0,11/20/20,E,2381555,32.522,3.712,3.45,None,19.4,19.34,19.31,...,19.31,19.33,3000,1800,1,0,0,1,0,2.0
1,11/20/20,J,2023138,None,None,None,1.99,105.15,105.06,103.7,...,104.55,104.82,800,800,0,0,0,0,0,1.6
2,11/20/20,L,2830137,63.162,-3.892,2.9,None,42.82,42.69,42.55,...,42.88,42.99,800,800,1,0,3,0,0,3.0
3,11/20/20,Z,23712821,18.52,-1.431,0.3,-23.17,109.49,109.25,109.92,...,108.55,108.88,1400,800,1,0,3,0,0,2.3
4,11/20/20,AE,17854,33.964,-0.519,None,None,26.33,25.91,25.91,...,26.18,26.99,900,900,0,0,0,0,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375891,06/08/20,ZGYHW,nan,nan,nan,nan,nan,0.1,nan,NaN,...,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,None
375892,06/08/20,ZIONN,nan,nan,nan,nan,nan,25.61,25.6,25.28,...,25.25,25.63,800,900,NaN,NaN,NaN,NaN,NaN,None
375893,06/08/20,ZIONO,nan,nan,nan,nan,nan,27.77,27.6,27.93,...,27.6,28.29,1000,1100,NaN,NaN,NaN,NaN,NaN,None
375894,06/08/20,ZIONP,990,nan,nan,nan,nan,21.01,20.97,21.0181,...,20.96,21.1,1200,1100,NaN,NaN,NaN,NaN,NaN,None


In [311]:
# sample = data.head(30000)
sample = data

In [312]:
sample = sample.astype({'open': float})

In [313]:
sample

,date,symbol,sharesShort,bookValue,trailingEps,forwardEps,pegRatio,previousClose,open,currentPrice,...,bid,ask,bidSize,askSize,strongBuy,buy,hold,sell,strongSell,recommendationMean
0,11/20/20,E,2381555,32.522,3.712,3.45,None,19.4,19.34,19.31,...,19.31,19.33,3000,1800,1,0,0,1,0,2.0
1,11/20/20,J,2023138,None,None,None,1.99,105.15,105.06,103.7,...,104.55,104.82,800,800,0,0,0,0,0,1.6
2,11/20/20,L,2830137,63.162,-3.892,2.9,None,42.82,42.69,42.55,...,42.88,42.99,800,800,1,0,3,0,0,3.0
3,11/20/20,Z,23712821,18.52,-1.431,0.3,-23.17,109.49,109.25,109.92,...,108.55,108.88,1400,800,1,0,3,0,0,2.3
4,11/20/20,AE,17854,33.964,-0.519,None,None,26.33,25.91,25.91,...,26.18,26.99,900,900,0,0,0,0,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375891,06/08/20,ZGYHW,nan,nan,nan,nan,nan,0.1,NaN,NaN,...,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,None
375892,06/08/20,ZIONN,nan,nan,nan,nan,nan,25.61,25.60,25.28,...,25.25,25.63,800,900,NaN,NaN,NaN,NaN,NaN,None
375893,06/08/20,ZIONO,nan,nan,nan,nan,nan,27.77,27.60,27.93,...,27.6,28.29,1000,1100,NaN,NaN,NaN,NaN,NaN,None
375894,06/08/20,ZIONP,990,nan,nan,nan,nan,21.01,20.97,21.0181,...,20.96,21.1,1200,1100,NaN,NaN,NaN,NaN,NaN,None


In [314]:
sample['dtd'] = sample.groupby('symbol')['open'].diff() * -1
sample

,date,symbol,sharesShort,bookValue,trailingEps,forwardEps,pegRatio,previousClose,open,currentPrice,...,ask,bidSize,askSize,strongBuy,buy,hold,sell,strongSell,recommendationMean,dtd
0,11/20/20,E,2381555,32.522,3.712,3.45,None,19.4,19.34,19.31,...,19.33,3000,1800,1,0,0,1,0,2.0,NaN
1,11/20/20,J,2023138,None,None,None,1.99,105.15,105.06,103.7,...,104.82,800,800,0,0,0,0,0,1.6,NaN
2,11/20/20,L,2830137,63.162,-3.892,2.9,None,42.82,42.69,42.55,...,42.99,800,800,1,0,3,0,0,3.0,NaN
3,11/20/20,Z,23712821,18.52,-1.431,0.3,-23.17,109.49,109.25,109.92,...,108.88,1400,800,1,0,3,0,0,2.3,NaN
4,11/20/20,AE,17854,33.964,-0.519,None,None,26.33,25.91,25.91,...,26.99,900,900,0,0,0,0,0,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375891,06/08/20,ZGYHW,nan,nan,nan,nan,nan,0.1,NaN,NaN,...,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,None,NaN
375892,06/08/20,ZIONN,nan,nan,nan,nan,nan,25.61,25.60,25.28,...,25.63,800,900,NaN,NaN,NaN,NaN,NaN,None,-0.4000
375893,06/08/20,ZIONO,nan,nan,nan,nan,nan,27.77,27.60,27.93,...,28.29,1000,1100,NaN,NaN,NaN,NaN,NaN,None,1.1500
375894,06/08/20,ZIONP,990,nan,nan,nan,nan,21.01,20.97,21.0181,...,21.1,1200,1100,NaN,NaN,NaN,NaN,NaN,None,-0.0199


In [315]:
sample['% dtd'] = sample["dtd"]/(sample['open'] - sample["dtd"]) * 100
sample

,date,symbol,sharesShort,bookValue,trailingEps,forwardEps,pegRatio,previousClose,open,currentPrice,...,bidSize,askSize,strongBuy,buy,hold,sell,strongSell,recommendationMean,dtd,% dtd
0,11/20/20,E,2381555,32.522,3.712,3.45,None,19.4,19.34,19.31,...,3000,1800,1,0,0,1,0,2.0,NaN,NaN
1,11/20/20,J,2023138,None,None,None,1.99,105.15,105.06,103.7,...,800,800,0,0,0,0,0,1.6,NaN,NaN
2,11/20/20,L,2830137,63.162,-3.892,2.9,None,42.82,42.69,42.55,...,800,800,1,0,3,0,0,3.0,NaN,NaN
3,11/20/20,Z,23712821,18.52,-1.431,0.3,-23.17,109.49,109.25,109.92,...,1400,800,1,0,3,0,0,2.3,NaN,NaN
4,11/20/20,AE,17854,33.964,-0.519,None,None,26.33,25.91,25.91,...,900,900,0,0,0,0,0,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375891,06/08/20,ZGYHW,nan,nan,nan,nan,nan,0.1,NaN,NaN,...,nan,nan,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
375892,06/08/20,ZIONN,nan,nan,nan,nan,nan,25.61,25.60,25.28,...,800,900,NaN,NaN,NaN,NaN,NaN,None,-0.4000,-1.538462
375893,06/08/20,ZIONO,nan,nan,nan,nan,nan,27.77,27.60,27.93,...,1000,1100,NaN,NaN,NaN,NaN,NaN,None,1.1500,4.347826
375894,06/08/20,ZIONP,990,nan,nan,nan,nan,21.01,20.97,21.0181,...,1200,1100,NaN,NaN,NaN,NaN,NaN,None,-0.0199,-0.094808


In [316]:
sample = sample.dropna()
sample

,date,symbol,sharesShort,bookValue,trailingEps,forwardEps,pegRatio,previousClose,open,currentPrice,...,bidSize,askSize,strongBuy,buy,hold,sell,strongSell,recommendationMean,dtd,% dtd
5192,11/19/20,AX,2857434,20.804,3.199,3.13,1.09,34.0,33.79,33.716,...,800,800,0,0,0,0,0,1.8,-0.41,-1.198830
5199,11/19/20,DX,1573794,18.528,7.583,1.82,-1.06,17.64,17.70,17.4042,...,1300,800,1,0,3,0,0,2.0,-0.13,-0.729108
5200,11/19/20,EC,7769367,8.82,2.14,1.0,-2.55,11.61,11.56,11.725,...,900,1000,0,0,8,4,0,2.5,0.20,1.760563
5201,11/19/20,ED,10015906,55.371,4.053,4.49,7.23,77.69,77.64,76.91,...,1000,800,1,0,7,5,1,3.4,-0.21,-0.269750
5207,11/19/20,GL,1112219,67.431,6.83,7.61,1.82,92.46,91.75,92.64,...,1000,1200,0,0,0,0,0,2.9,1.23,1.358816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365457,06/10/20,ZGYHR,nan,nan,nan,nan,nan,0.121,0.13,NaN,...,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,-0.00,-0.000000
365458,06/10/20,ZGYHU,nan,nan,nan,nan,nan,9.86,9.80,9.8,...,2900,3100,NaN,NaN,NaN,NaN,NaN,nan,-0.00,-0.000000
365460,06/10/20,ZIONN,nan,nan,nan,nan,nan,25.53,25.16,25.15,...,800,900,NaN,NaN,NaN,NaN,NaN,nan,-0.09,-0.356436
365461,06/10/20,ZIONO,nan,nan,nan,nan,nan,27.75,28.00,28.1385,...,1000,1100,NaN,NaN,NaN,NaN,NaN,nan,-0.49,-1.719902


In [317]:
sample['timedelta'] = [(datetime.datetime.strptime("11/20/20", '%m/%d/%y') - datetime.datetime.strptime(x, '%m/%d/%y')).days for x in sample['date']]

/Users/johnschulz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [318]:
X1 = sample
X1

,date,symbol,sharesShort,bookValue,trailingEps,forwardEps,pegRatio,previousClose,open,currentPrice,...,askSize,strongBuy,buy,hold,sell,strongSell,recommendationMean,dtd,% dtd,timedelta
5192,11/19/20,AX,2857434,20.804,3.199,3.13,1.09,34.0,33.79,33.716,...,800,0,0,0,0,0,1.8,-0.41,-1.198830,1
5199,11/19/20,DX,1573794,18.528,7.583,1.82,-1.06,17.64,17.70,17.4042,...,800,1,0,3,0,0,2.0,-0.13,-0.729108,1
5200,11/19/20,EC,7769367,8.82,2.14,1.0,-2.55,11.61,11.56,11.725,...,1000,0,0,8,4,0,2.5,0.20,1.760563,1
5201,11/19/20,ED,10015906,55.371,4.053,4.49,7.23,77.69,77.64,76.91,...,800,1,0,7,5,1,3.4,-0.21,-0.269750,1
5207,11/19/20,GL,1112219,67.431,6.83,7.61,1.82,92.46,91.75,92.64,...,1200,0,0,0,0,0,2.9,1.23,1.358816,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365457,06/10/20,ZGYHR,nan,nan,nan,nan,nan,0.121,0.13,NaN,...,nan,NaN,NaN,NaN,NaN,NaN,NaN,-0.00,-0.000000,163
365458,06/10/20,ZGYHU,nan,nan,nan,nan,nan,9.86,9.80,9.8,...,3100,NaN,NaN,NaN,NaN,NaN,nan,-0.00,-0.000000,163
365460,06/10/20,ZIONN,nan,nan,nan,nan,nan,25.53,25.16,25.15,...,900,NaN,NaN,NaN,NaN,NaN,nan,-0.09,-0.356436,163
365461,06/10/20,ZIONO,nan,nan,nan,nan,nan,27.75,28.00,28.1385,...,1100,NaN,NaN,NaN,NaN,NaN,nan,-0.49,-1.719902,163


In [319]:
X1 = X1[(X1 != 'nan').all(1)]

/Users/johnschulz/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [320]:
X1 = X1[(X1 != 'NaN').all(1)]

In [321]:
X1 = X1[(X1 != '{}').all(1)]

In [322]:
X1 = X1[~X1.isin([np.nan, np.inf, -np.inf]).any(1)]

In [323]:
X = X1.drop(columns=['% dtd', 'dtd', 'date', 'symbol'])

In [324]:
X

,sharesShort,bookValue,trailingEps,forwardEps,pegRatio,previousClose,open,currentPrice,beta,trailingPE,...,ask,bidSize,askSize,strongBuy,buy,hold,sell,strongSell,recommendationMean,timedelta
5192,2857434,20.804,3.199,3.13,1.09,34.0,33.790,33.716,1.751409,10.539544,...,33.95,800,800,0,0,0,0,0,1.8,1
5199,1573794,18.528,7.583,1.82,-1.06,17.64,17.700,17.4042,1.307215,2.29516,...,17.41,1300,800,1,0,3,0,0,2.0,1
5200,7769367,8.82,2.14,1.0,-2.55,11.61,11.560,11.725,1.11251,5.478972,...,11.72,900,1000,0,0,8,4,0,2.5,1
5201,10015906,55.371,4.053,4.49,7.23,77.69,77.640,76.91,0.15558,18.976068,...,77.1,1000,800,1,0,7,5,1,3.4,1
5207,1112219,67.431,6.83,7.61,1.82,92.46,91.750,92.64,1.11443,13.563689,...,92.4,1000,1200,0,0,0,0,0,2.9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364775,10733949,24.034,5.522,6.72,2.08,110.76,111.110,110.49,0.622724,20.009054,...,110.56,800,800,8,5,22,0,1,2.9,163
364776,12697795,28.262,2.287,2.15,1.74,24.08,23.390,23.14,1.7369,10.118058,...,23.19,1000,3200,1,2,22,2,1,2.2,163
364778,1549306,60.112,11.951,18.73,6.44,1039.47,1042.620,1026.94,1.214561,85.92921,...,1025.51,1000,1000,3,4,23,4,1,2.3,163
364779,28765103,14.551,4.43,4.88,6.41,57.86,57.895,57.93,0.423607,13.07675,...,57.93,1800,900,2,6,24,0,0,2.6,163


In [325]:
X1

,date,symbol,sharesShort,bookValue,trailingEps,forwardEps,pegRatio,previousClose,open,currentPrice,...,askSize,strongBuy,buy,hold,sell,strongSell,recommendationMean,dtd,% dtd,timedelta
5192,11/19/20,AX,2857434,20.804,3.199,3.13,1.09,34.0,33.790,33.716,...,800,0,0,0,0,0,1.8,-0.410,-1.198830,1
5199,11/19/20,DX,1573794,18.528,7.583,1.82,-1.06,17.64,17.700,17.4042,...,800,1,0,3,0,0,2.0,-0.130,-0.729108,1
5200,11/19/20,EC,7769367,8.82,2.14,1.0,-2.55,11.61,11.560,11.725,...,1000,0,0,8,4,0,2.5,0.200,1.760563,1
5201,11/19/20,ED,10015906,55.371,4.053,4.49,7.23,77.69,77.640,76.91,...,800,1,0,7,5,1,3.4,-0.210,-0.269750,1
5207,11/19/20,GL,1112219,67.431,6.83,7.61,1.82,92.46,91.750,92.64,...,1200,0,0,0,0,0,2.9,1.230,1.358816,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364775,06/10/20,CHKP,10733949,24.034,5.522,6.72,2.08,110.76,111.110,110.49,...,800,8,5,22,0,1,2.9,-1.870,-1.655160,163
364776,06/10/20,FITB,12697795,28.262,2.287,2.15,1.74,24.08,23.390,23.14,...,3200,1,2,22,2,1,2.2,-2.820,-10.759252,163
364778,06/10/20,CMG,1549306,60.112,11.951,18.73,6.44,1039.47,1042.620,1026.94,...,1000,3,4,23,4,1,2.3,-40.230,-3.715196,163
364779,06/10/20,VZ,28765103,14.551,4.43,4.88,6.41,57.86,57.895,57.93,...,900,2,6,24,0,0,2.6,-0.415,-0.711713,163


In [326]:
X = X.apply(pd.to_numeric)

In [327]:
keep = (X <= 100000000000).all(1)

In [328]:
X = X[keep]

In [329]:
X1 = X1[keep]

In [356]:
winner = X1['% dtd'] > 10

In [357]:
y = list(X1['% dtd'])
y = winner

In [358]:
X

,index,sharesShort,bookValue,trailingEps,forwardEps,pegRatio,previousClose,open,currentPrice,beta,...,ask,bidSize,askSize,strongBuy,buy,hold,sell,strongSell,recommendationMean,timedelta
0,5192,2857434,20.804,3.199,3.13,1.09,34.00,33.790,33.7160,1.751409,...,33.95,800,800,0,0,0,0,0,1.8,1
1,5199,1573794,18.528,7.583,1.82,-1.06,17.64,17.700,17.4042,1.307215,...,17.41,1300,800,1,0,3,0,0,2.0,1
2,5200,7769367,8.820,2.140,1.00,-2.55,11.61,11.560,11.7250,1.112510,...,11.72,900,1000,0,0,8,4,0,2.5,1
3,5201,10015906,55.371,4.053,4.49,7.23,77.69,77.640,76.9100,0.155580,...,77.10,1000,800,1,0,7,5,1,3.4,1
4,5207,1112219,67.431,6.830,7.61,1.82,92.46,91.750,92.6400,1.114430,...,92.40,1000,1200,0,0,0,0,0,2.9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117861,364775,10733949,24.034,5.522,6.72,2.08,110.76,111.110,110.4900,0.622724,...,110.56,800,800,8,5,22,0,1,2.9,163
117862,364776,12697795,28.262,2.287,2.15,1.74,24.08,23.390,23.1400,1.736900,...,23.19,1000,3200,1,2,22,2,1,2.2,163
117863,364778,1549306,60.112,11.951,18.73,6.44,1039.47,1042.620,1026.9400,1.214561,...,1025.51,1000,1000,3,4,23,4,1,2.3,163
117864,364779,28765103,14.551,4.430,4.88,6.41,57.86,57.895,57.9300,0.423607,...,57.93,1800,900,2,6,24,0,0,2.6,163


In [359]:
X.dropna()

,index,sharesShort,bookValue,trailingEps,forwardEps,pegRatio,previousClose,open,currentPrice,beta,...,ask,bidSize,askSize,strongBuy,buy,hold,sell,strongSell,recommendationMean,timedelta
0,5192,2857434,20.804,3.199,3.13,1.09,34.00,33.790,33.7160,1.751409,...,33.95,800,800,0,0,0,0,0,1.8,1
1,5199,1573794,18.528,7.583,1.82,-1.06,17.64,17.700,17.4042,1.307215,...,17.41,1300,800,1,0,3,0,0,2.0,1
2,5200,7769367,8.820,2.140,1.00,-2.55,11.61,11.560,11.7250,1.112510,...,11.72,900,1000,0,0,8,4,0,2.5,1
3,5201,10015906,55.371,4.053,4.49,7.23,77.69,77.640,76.9100,0.155580,...,77.10,1000,800,1,0,7,5,1,3.4,1
4,5207,1112219,67.431,6.830,7.61,1.82,92.46,91.750,92.6400,1.114430,...,92.40,1000,1200,0,0,0,0,0,2.9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117861,364775,10733949,24.034,5.522,6.72,2.08,110.76,111.110,110.4900,0.622724,...,110.56,800,800,8,5,22,0,1,2.9,163
117862,364776,12697795,28.262,2.287,2.15,1.74,24.08,23.390,23.1400,1.736900,...,23.19,1000,3200,1,2,22,2,1,2.2,163
117863,364778,1549306,60.112,11.951,18.73,6.44,1039.47,1042.620,1026.9400,1.214561,...,1025.51,1000,1000,3,4,23,4,1,2.3,163
117864,364779,28765103,14.551,4.430,4.88,6.41,57.86,57.895,57.9300,0.423607,...,57.93,1800,900,2,6,24,0,0,2.6,163


In [360]:
import numpy as np

In [361]:
X =X.reset_index()

In [362]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [363]:
y_train

341195    False
218330    False
175452    False
109225    False
59478     False
          ...  
243680    False
343377    False
325329    False
8115      False
55304     False
Name: % dtd, Length: 78970, dtype: bool

In [364]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [365]:
y_predicted =model.predict(X_test)
scores = model.score(X_test, y_test)

In [366]:
print(y_predicted)
print(scores)

[False False False ... False False False]
0.9752673796791443


In [367]:
y_predicted

array([False, False, False, ..., False, False, False])

In [368]:
from sklearn import metrics

In [369]:
metrics.accuracy_score(y_test,list(y_predicted))

0.9752673796791443

In [370]:
y_predicted

array([False, False, False, ..., False, False, False])

In [371]:
list(y_predicted)

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
